In [ ]:
# Package Imports
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil


ipylab module is not installed, menus and toolbar are disabled.


In [2]:
# Helper and script imports
import sys
sys.path.append("/Users/emilieyu/endotehelial-masboss")
from boolean_models.scripts.run_single import main
from boolean_models.analysis import (
    generate_ko_models,
    compute_delta,
    classify_phenotype,
)

In [ ]:
# Set project root dir
PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
MODELS_DIR = PROJECT_ROOT / "boolean_models" / "models"
RESULTS_DIR = PROJECT_ROOT / "results" / "boolean_models"

# load model and WT configuration
model_file = MODELS_DIR / "rho.bnd"
cfg_file   = MODELS_DIR / "rho_base.cfg"

model = maboss.load(str(model_file), str(cfg_file))
nodes = ['DSP', 'TJP1', 'JCAD']

### Basic Perturbation Behaviour Validation

Generate perturbation scenarios, run perturbations, and store raw output

In [5]:
ko_scenarios = generate_ko_models(model, nodes)

name: DSP_ko for combo ('DSP',)
name: TJP1_ko for combo ('TJP1',)
name: JCAD_ko for combo ('JCAD',)
name: DSP_TJP1_ko for combo ('DSP', 'TJP1')
name: DSP_JCAD_ko for combo ('DSP', 'JCAD')
name: TJP1_JCAD_ko for combo ('TJP1', 'JCAD')
name: DSP_TJP1_JCAD_ko for combo ('DSP', 'TJP1', 'JCAD')


In [6]:
def save_sim_details(result, result_directory, perb_name):
    # Save node probabilities
    nodes_probtraj = result.get_nodes_probtraj().rename_axis('t').reset_index()    
    nodes_probtraj.to_csv(
        result_directory / f"{perb_name}_nodes_probtraj.csv",
        index=False
    )

    # Save state probabilities
    states_probtraj = result.get_states_probtraj().rename_axis('t').reset_index()    
    states_probtraj.to_csv(
        result_directory / f"{perb_name}_states_probtraj.csv",
        index=False
    )


In [7]:
perb_dfs = {}
for name, model in ko_scenarios.items():
    res_dir = RESULTS_DIR / "rho_model" / name
    
    if res_dir.exists():
        shutil.rmtree(res_dir)

    res_dir.mkdir(parents=True, exist_ok=True)

    m = model
    res = m.run()

    save_sim_details(res, res_dir, name)
    
    prob_df = res.get_nodes_probtraj()
    balance_df = prob_df.copy()
    balance_df["delta"] = prob_df["RhoC"] - prob_df["RhoA"]

    perb_dfs[name] = balance_df


In [8]:
perb_dfs['TJP1_ko']

,DSP,JCAD,RhoA,RhoC,TJP1,delta
0.0,0.503999,0.491500,0.535193,0.489053,0.0,-0.046140
0.1,0.504000,0.491500,0.584433,0.461833,0.0,-0.122600
0.2,0.504000,0.491500,0.610293,0.439491,0.0,-0.170802
0.3,0.504000,0.491500,0.624020,0.426303,0.0,-0.197717
0.4,0.504001,0.491500,0.634082,0.415887,0.0,-0.218195
0.5,0.503999,0.491500,0.646775,0.408705,0.0,-0.238070
0.6,0.504002,0.491500,0.653065,0.407230,0.0,-0.245835
0.7,0.504000,0.491500,0.658439,0.404085,0.0,-0.254354
0.8,0.504001,0.491500,0.662142,0.402754,0.0,-0.259388
0.9,0.504001,0.491500,0.663247,0.399170,0.0,-0.264077
